In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding

In [ ]:
raw_datasers=load_dataset('glue','mrpc')
model='bert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_fun(batch):
  return tokenizer(batch['sentence1'],batch['sentence2'],truncation=True)

In [ ]:
tokenized_data=raw_datasers.map(tokenize_fun,batched=True)
data_col=DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
tokenized_data=tokenized_data.remove_columns(['sentence1','sentence2','idx'])
tokenized_data=tokenized_data.rename_column('label','labels')
tokenized_data.set_format('torch')

In [ ]:
tokenized_data['test'].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train=DataLoader(
    tokenized_data['train'],shuffle=True,batch_size=8,collate_fn=data_col
)
eval=DataLoader(
    tokenized_data['validation'],shuffle=True,batch_size=8,collate_fn=data_col
    )

In [ ]:
for batch in train:
  break

In [ ]:
{k:v.shape for k,v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 68]),
 'token_type_ids': torch.Size([8, 68]),
 'attention_mask': torch.Size([8, 68])}

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
mod=AutoModelForSequenceClassification.from_pretrained(model,num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs=mod(**batch)
outputs.loss,outputs.logits.shape

(tensor(0.7373, grad_fn=<NllLossBackward0>), torch.Size([8, 2]))

In [ ]:
from torch.optim import AdamW
optim=AdamW(mod.parameters(),lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs=3
num_train_steps=num_epochs*len(train)
lr_scheduler=get_scheduler('linear',optimizer=optim,num_warmup_steps=0,num_training_steps=num_train_steps)

In [ ]:
num_train_steps

1377

In [ ]:
import torch

In [ ]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
mod.to(device)
device

device(type='cpu')

In [ ]:
from tqdm.auto import tqdm

progress=tqdm(range(num_train_steps))

mod.train()
for epoch in range(num_epochs):
  for batch in train:
    batch ={k:v.to(device) for k,v in batch.items()}
    outputs=mod(**batch)
    loss=outputs.loss
    loss.backward()
    optim.step()
    lr_scheduler.step()
    optim.zero_grad()
    progress.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
pip install evaluate

In [ ]:
import evaluate

In [ ]:
metric=evaluate.load('glue','mrpc')
mod.eval()
for batch in eval:
  batch={k:v.to(device) for k,v in batch.items()}
  with torch.inference_mode():
    outputs=mod(**batch)
  logits=outputs.logits
  preds=torch.argmax(logits,dim=-1)
  metric.add_batch(predictions=preds,references=batch['labels'])
metric.compute()

In [ ]:
from accelerate import Accelerator
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification,get_scheduler

In [ ]:
accel=Accelerator()

model1=AutoModelForSequenceClassification.from_pretrained(model,num_labels=2)
optim=AdamW(model1.parameters(),lr=3e-5)

In [ ]:
train_dl,eval_dl,model1,optim=accel.prepare(
    train,eval,model1,optim
)

In [ ]:
num_epochs=3
num_train_steps=num_epochs*len(train_dl)
lr_scheduler=get_scheduler(
    'linear',
    optimizer=optim,
    num_warmup_steps=0,
    num_training_steps=num_train_steps
)

In [ ]:
progress=tqdm(range(num_train_steps))

model1.train()
for epoch in range(num_epochs):
  for batch in train_dl:
    outputs=model1(**batch)
    loss=outputs.loss
    accel.backward(loss)

    optim.step()
    lr_scheduler.step()
    optim.zero_grad()
    progress.update(1)

In [ ]:
from accelerate import notebook_launcher
notebook_launcher(train)